In [1]:
import os
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [2]:
import torch.multiprocessing as mp
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

## basics

- All to one：reduce； one to All：broadcast
- rank：`[0, world_size-1]`

## ddp setup

In [4]:
world_size = torch.cuda.device_count()
world_size

2

In [3]:
def ddp_setup(rank, world_size):
    """
    Args:
        rank: Unique identifier of each process
        world_size: Total number of processes
    """
    # rank 0 process
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "12355"
    # nccl：NVIDIA Collective Communication Library 
    # 分布式情况下的，gpus 间通信
    init_process_group(backend="nccl", rank=rank, world_size=world_size)
    torch.cuda.set_device(rank)

## custom Trainer（自定义 trainer）

In [5]:
class Trainer:
    def __init__(self, 
                 model: torch.nn.Module, 
                 train_dataloader: DataLoader, 
                 optimizer: torch.optim.Optimizer, 
                 gpu_id: int) -> None:
        # rank
        self.gpu_id = gpu_id
        self.model = model.to(gpu_id)
        self.train_dataloader = train_dataloader
        self.optimizer = optimizer
        self.model = DDP(model, device_ids=[gpu_id])
    
    def _run_batch(self, xs, ys):
        self.optimizer.zero_grad()
        output = self.model(xs)
        loss = F.cross_entropy(output, ys)
        loss.backward()
        self.optimizer.step()
    
    def _run_epoch(self, epoch):
        batch_size = len(next(iter(self.train_dataloader))[0])
        print(f'[GPU: {self.gpu_id}] Epoch: {epoch} | Batchsize: {batch_size} | Steps: {len(self.train_dataloader)}')
        self.train_dataloader.sampler.set_epoch(epoch)
        for xs, ys in self.train_dataloader:
            xs = xs.to(self.gpu_id)
            ys = ys.to(self.gpu_id)
            self._run_batch(xs, ys)
    
    def train(self, max_epoch: int):
        for epoch in range(max_epoch):
            self._run_epoch(epoch)

## pipeline

In [6]:
class MyTrainDataset(Dataset):
    def __init__(self, size):
        self.size = size
        self.data = [(torch.rand(20), torch.rand(1)) for _ in range(size)]

    def __len__(self):
        return self.size
    
    def __getitem__(self, index):
        return self.data[index]

In [7]:
train_dataset = MyTrainDataset(2048)

In [8]:
train_dataset[0]

(tensor([0.4790, 0.5080, 0.1934, 0.5247, 0.6372, 0.9930, 0.2379, 0.9182, 0.3659,
         0.8408, 0.2347, 0.1770, 0.8691, 0.2810, 0.2156, 0.8289, 0.9372, 0.6358,
         0.4338, 0.2754]),
 tensor([0.6307]))

In [ ]:
def main(rank: int, world_size: int, max_epochs: int, batch_size: int):
    ddp_setup(rank, world_size)
    
    train_dataset = MyTrainDataset(2048)
    train_dataloader = DataLoader(train_dataset, 
                              batch_size=batch_size, 
                              pin_memory=True, 
                              shuffle=False, 
                              # batch input: split to each gpus (且没有任何 overlaping samples 各个 gpu 之间)
                              sampler=DistributedSampler(train_dataset))
    model = torch.nn.Linear(20, 1)
    optimzer = torch.optim.SGD(model.parameters(), lr=1e-3)
    
    trainer = Trainer(model=model, gpu_id=rank, optimizer=optimzer, train_dataloader=train_dataloader)
    trainer.train(max_epochs)
    
    destroy_process_group()

## 分布式 training

In [9]:
world_size = torch.cuda.device_count()
world_size

2

In [11]:
2048/32

64.0

In [13]:
(32*32) * 2

2048

In [10]:
# mp.spawn(main, args=(world_size, 10, 32), nprocs=world_size)
!python ddp_gpus.py --max_epochs 5 --batch_size 32 

[GPU: 0] Epoch: 0 | Batchsize: 32 | Steps: 32
[GPU: 1] Epoch: 0 | Batchsize: 32 | Steps: 32
[GPU: 0] Epoch: 1 | Batchsize: 32 | Steps: 32
[GPU: 1] Epoch: 1 | Batchsize: 32 | Steps: 32
[GPU: 0] Epoch: 2 | Batchsize: 32 | Steps: 32
[GPU: 1] Epoch: 2 | Batchsize: 32 | Steps: 32
[GPU: 1] Epoch: 3 | Batchsize: 32 | Steps: 32
[GPU: 0] Epoch: 3 | Batchsize: 32 | Steps: 32
[GPU: 0] Epoch: 4 | Batchsize: 32 | Steps: 32
[GPU: 1] Epoch: 4 | Batchsize: 32 | Steps: 32


## torchrun 运行

- https://pytorch.org/docs/stable/elastic/run.html
- fault-tolerant 
- `torchrun`
    - Worker RANK and WORLD_SIZE are assigned automatically.
    - `python -m torch.distributed.launch --use-env`

In [14]:
!torchrun ddp_gpus_torchrun.py --max_epochs 5 --batch_size 32

[GPU: 0] Epoch: 0 | Batchsize: 32 | Steps: 64
[GPU: 0] Epoch: 1 | Batchsize: 32 | Steps: 64
[GPU: 0] Epoch: 2 | Batchsize: 32 | Steps: 64
[GPU: 0] Epoch: 3 | Batchsize: 32 | Steps: 64
[GPU: 0] Epoch: 4 | Batchsize: 32 | Steps: 64


In [15]:
!torchrun --nproc-per-node=2 ddp_gpus_torchrun.py --max_epochs 5 --batch_size 32

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
[GPU: 0] Epoch: 0 | Batchsize: 32 | Steps: 32
[GPU: 1] Epoch: 0 | Batchsize: 32 | Steps: 32
[GPU: 0] Epoch: 1 | Batchsize: 32 | Steps: 32[GPU: 1] Epoch: 1 | Batchsize: 32 | Steps: 32

[GPU: 0] Epoch: 2 | Batchsize: 32 | Steps: 32
[GPU: 1] Epoch: 2 | Batchsize: 32 | Steps: 32
[GPU: 1] Epoch: 3 | Batchsize: 32 | Steps: 32
[GPU: 0] Epoch: 3 | Batchsize: 32 | Steps: 32
[GPU: 1] Epoch: 4 | Batchsize: 32 | Steps: 32
[GPU: 0] Epoch: 4 | Batchsize: 32 | Steps: 32


In [16]:
!python -m torch.distributed.launch --use-env --nproc-per-node=2 ddp_gpus_torchrun.py --max_epochs 5 --batch_size 32

/home/whaow/anaconda3/lib/python3.10/site-packages/torch/distributed/launch.py:181: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
[GPU: 0] Epoch: 0 | Batchsize: 32 | Steps: 32[GPU: 1] Epoch: 0 | Batchsize: 32 | Steps: 32

[GPU: 0] Epoch: 1 | Batchsize: 32 | Steps: 32
[GPU: 1] Epoch: 1 | Batchsize: 32 | Steps: 32
[GPU: 0] Epoch: 2 | Batchsize: 32 | Step